#  11. Solidity 제어, 데이터구조

* Last Updated 20200606SAT1200 20190120SUN0000 20180707SAT1300 20170112
* todo: 1) ```watch``` 2) library ```myLib+StringUtils``` 3) 계정에서 발생한 거래의 목록? 4) registry

## 9.1 학습내용

### 9.1.1 목표
 
* solidity 배우기

### 9.1.2 목차

* 9.6 [제어](#9.6)
    * 문제 Hello World Contract - 이름정하고, 헬로우 할 때 알아내기
* 9.7 데이터구조
    * 9.7.1 Array
    * 9.7.2 [mapping](#S.5.2)
    * 실습 Members
    * 실습 BankV4
* 9.8 [Exception](#9.6)


## 9.6 제어
<a id='11.4'></a>


### 9.6.1 반복문
블록체인에서의 프로그램은 명령문마다 비용이 발생하는 특성을 가지고 있기 때문에, 반복을 하게되면 비용이 발생하게 된다.
gas가 반복마다 소비되고 한도 gasLimit를 초과하게 되면 실행이 중지될 수 밖에 없다.

* for문
```python
function getSum() public returns(uint) {
    uint sum=0;
    for(uint i=0;i<10;i++)
        sum+=i;
    return sum;
}
```

* while문
```python
function getSum() public returns(uint) {
    uint sum=0;
    uint i=0;
    do {
        sum+=i;
        i++;
    } while(i<10);
    return sum;
}
```

반복횟수에 따라 gas비용이 급증하는 것을 볼 수 있다.
특히 sum 변수를 메모리에 두는 경우에 비해, storage에 (state variable) 두는 경우 비용이 매우 높았다.
메모리를 사용하는 것에 비해 Storage가 많이 비용이 많이 든다.

반복 횟수 | sum을 로컬에 두는 경우 | sum을 state variable에 두는 경우
-----|-----|-----
10 | 약 30,000 (23845 + 6773)  |  약 64,000 (43045 + 21773)
100 | 약 78,000 (49345 + 28073)  |  약 368,000 (192881 + 175809)
1000 | 약 203,000 (112345 + 91073) | 약 3,408,000 (1712981 + 1695909)
10000 | REMIX가 죽고, 다시 실행 | 좌동

In [10]:
%%writefile src/ControlFlowTest.sol
pragma solidity ^0.6.0;
contract ControlFlowTest {
    uint sum;
    constructor() public {
        sum = 0;
    }
    function doForLoop() public {
        //sum = 0;
        uint sumLocal = 0;
        for(uint i = 0; i < 101; i++) {
            sumLocal += i;
        }
        sum = sumLocal;
    }
    function doForLoop2() public {
        sum = 0;
        //uint sumLocal = 0;
        for(uint i = 0; i < 101; i++) {
            //sumLocal += i;
            sum += i;
        }
        //sum = sumLocal;
    }
    function getSum() view public returns(uint) {
        return sum;
    }
}

Writing src/ControlFlowTest.sol


* switch, goto를 제외하고, 다른 언어에서 제공하는 if, else, while, do, for, break, continue, return, ?

```python
if(condition) {
    ...
} else if {
    ...
}
```

* 삼항 연산자 ternary operator
```python
betAmount=betAmount>amount ? amount : betAmount;
```

## 실습: 인사하는 컨트랙

"Hello"를 출력하는 프로그램을 수정해서 if문을 사용해 보자.
modifier를 사용하면 조건문을 사용하지 않을 수 있다.
조건문에서 비교문을 사용하게 되는데, ```string```은 참조타입이라 비교를 하면 주소를 비교하게 된다. 어떻게 해야 하는지 생각해보자.

```python
contract Hello1 {
    function sayHello() 
    modifier isOwner()
    function setHello()
    function compareTo(string memory _str)
    function getBalance()
}
```

###  단계 1: 컨트랙 개발

Line | 설명
-----|-----
1 | major version 6 이상 최신으로 컴파일
2 | contract 명. 파일명과 반드시 일치할 필요가 없다.
3 | 속성 ```hello```. 누구나 사용할 수 없도록 ```private```
4 | 속성 ```address```. ```private```으로 자신만 사용할 수 있게 선언.
5 | ```event```로서 주소, 문자열 인자를 출력한다. Solidity에는 print문이 별도로 없다. 문자열을 반환하거나 ```event```문으로 출력할 수 있다.
6 ~ 9 | 생성자로서 외부에서 인자를 받아 속성을 초기화. ```constructor```는 ```public``` 또는 ```internal```로 선언해야 한다. 속성 값을 넣어준다. ```msg```는 전역변수로서 ```msg.sender```는 전송자의 계정주소를 말한다. 생성자의 ```msg.sender```를 컨트랙의 소유주 ```owner```로 등록한다.
10 ~ 12 | ```sayHello()``` 함수는 속성 값을 읽는다. 따라서 ```view```로 선언한다. 인자는 함수 내에서만 잠시 사용하므로 ```memory```로 선언
13 ~ 18 | ```modifier``` 함수. ```msg.sender```와 ```owner```가 일치하지 않으면 예외처리한다. 아니면 계속 실행.
19 ~ 29 | ```setHello()``` 함수는 ```event```함수를 가지고 있으므로 ```view```로 선언하지 않는다. ```view```와 ```event```는 같이 사용할 수 없다. ```event```는 상태를 변경하기 때문이다.
30 ~ 37 | ```compareTo()```함수는 입력문자열과 hello를 비교한다. 문자열은 참조값이기 때문에 비교할 경우 참조를 비교하게 되므로 주의를 해야한다. hash로 비교하고 있다.
38 ~ 40 | 잔고를 구한다.


In [2]:
%%writefile src/Hello1.sol
pragma solidity ^0.6.0;
contract Hello1 {
    string private hello;
    address private owner;
    event PrintLog(address addr, string s);
    constructor(string memory _hello) public {
        hello = _hello;
        owner = msg.sender;
    }
    function sayHello() view public returns(string memory) {
        return hello;
    }
    modifier isOwner() {
        if (msg.sender != owner) {
            revert();
        }
        _; //continue executing rest of method body
    }
    function setHello() public {
        string memory s = "";
        if (msg.sender == owner) {
            s = "Hello";
            emit PrintLog(msg.sender, s);
        } else {
            s = "Olleh";
            emit PrintLog(msg.sender, s);
        }
        hello = s;
    }
    function compareTo(string memory _str) view public returns(bool) {
        bool isEqual = false;
        //if (hello == _str) {
        if (keccak256(bytes(hello)) == keccak256(bytes(_str))) {
            isEqual = true;
        }
        return isEqual;
    }
    function getBalance() view public isOwner returns(uint) {
        return address(this).balance;
    }
}


Overwriting src/Hello1.sol


### 단계 2: 컴파일

In [3]:
!solc --abi --bin --gas src/Hello1.sol


======= src/Hello1.sol:Hello1 =======
Gas estimation:
construction:
   infinite + 349800 = infinite
external:
   compareTo(string):	infinite
   getBalance():	1063
   sayHello():	infinite
   setHello():	infinite
Binary:
608060405234801561001057600080fd5b506040516108d53803806108d58339818101604052602081101561003357600080fd5b810190808051604051939291908464010000000082111561005357600080fd5b8382019150602082018581111561006957600080fd5b825186600182028301116401000000008211171561008657600080fd5b8083526020830192505050908051906020019080838360005b838110156100ba57808201518184015260208101905061009f565b50505050905090810190601f1680156100e75780820380516001836020036101000a031916815260200191505b50604052505050806000908051906020019061010492919061014c565b5033600160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550506101f1565b828054600181600116156101000203166002900490600052602060002090601f016020900481019282601f1061018d57805160ff1916838001

### 단계 3: 배포

bytecode는 16진수이므로 앞에 ```0x```를 붙여준다.
gas를 지급해야 하는 등 비용이 발생하는 경우에는 반드시 web3.personal.unlockAccount()를 해주어야 한다.
REMIX를 사용하면, Web3 deploy를 복사해서 사용할 수 있다. abi, bin을 넣은 javascript명령문을 발견할 수 있다.

In [7]:
%%writefile src/Hello1Deploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8345"));
//var _abiArray = JSON.parse(_abiStr);
var _abiArray = [{"inputs":[{"internalType":"string","name":"_hello","type":"string"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"addr","type":"address"},{"indexed":false,"internalType":"string","name":"s","type":"string"}],"name":"PrintLog","type":"event"},{"inputs":[{"internalType":"string","name":"_str","type":"string"}],"name":"compareTo","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"sayHello","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"setHello","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var _bin = "0x" + "608060405234801561001057600080fd5b506040516108d53803806108d58339818101604052602081101561003357600080fd5b810190808051604051939291908464010000000082111561005357600080fd5b8382019150602082018581111561006957600080fd5b825186600182028301116401000000008211171561008657600080fd5b8083526020830192505050908051906020019080838360005b838110156100ba57808201518184015260208101905061009f565b50505050905090810190601f1680156100e75780820380516001836020036101000a031916815260200191505b50604052505050806000908051906020019061010492919061014c565b5033600160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550506101f1565b828054600181600116156101000203166002900490600052602060002090601f016020900481019282601f1061018d57805160ff19168380011785556101bb565b828001600101855582156101bb579182015b828111156101ba57825182559160200191906001019061019f565b5b5090506101c891906101cc565b5090565b6101ee91905b808211156101ea5760008160009055506001016101d2565b5090565b90565b6106d5806102006000396000f3fe608060405234801561001057600080fd5b506004361061004c5760003560e01c806312065fe01461005157806327906b251461006f57806373fde99a14610142578063ef5fb05b1461014c575b600080fd5b6100596101cf565b6040518082815260200191505060405180910390f35b6101286004803603602081101561008557600080fd5b81019080803590602001906401000000008111156100a257600080fd5b8201836020820111156100b457600080fd5b803590602001918460018302840111640100000000831117156100d657600080fd5b91908080601f016020809104026020016040519081016040528093929190818152602001838380828437600081840152601f19601f820116905080830192505050505050509192919290505050610231565b604051808215151515815260200191505060405180910390f35b61014a6102be565b005b610154610558565b6040518080602001828103825283818151815260200191508051906020019080838360005b83811015610194578082015181840152602081019050610179565b50505050905090810190601f1680156101c15780820380516001836020036101000a031916815260200191505b509250505060405180910390f35b6000600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff161461022b57600080fd5b47905090565b6000806000905082805190602001206000604051808280546001816001161561010002031660029004801561029d5780601f1061027b57610100808354040283529182019161029d565b820191906000526020600020905b815481529060010190602001808311610289575b5050915050604051809103902014156102b557600190505b80915050919050565b6060604051806020016040528060008152509050600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff161415610435576040518060400160405280600581526020017f48656c6c6f00000000000000000000000000000000000000000000000000000081525090507fd2065d4a97074e981ab7a6ff96587a468761d4c023459e930236e2f49ef6f64f3382604051808373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200180602001828103825283818151815260200191508051906020019080838360005b838110156103f55780820151818401526020810190506103da565b50505050905090810190601f1680156104225780820380516001836020036101000a031916815260200191505b50935050505060405180910390a161053e565b6040518060400160405280600581526020017f4f6c6c656800000000000000000000000000000000000000000000000000000081525090507fd2065d4a97074e981ab7a6ff96587a468761d4c023459e930236e2f49ef6f64f3382604051808373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200180602001828103825283818151815260200191508051906020019080838360005b838110156105025780820151818401526020810190506104e7565b50505050905090810190601f16801561052f5780820380516001836020036101000a031916815260200191505b50935050505060405180910390a15b80600090805190602001906105549291906105fa565b5050565b606060008054600181600116156101000203166002900480601f0160208091040260200160405190810160405280929190818152602001828054600181600116156101000203166002900480156105f05780601f106105c5576101008083540402835291602001916105f0565b820191906000526020600020905b8154815290600101906020018083116105d357829003601f168201915b5050505050905090565b828054600181600116156101000203166002900490600052602060002090601f016020900481019282601f1061063b57805160ff1916838001178555610669565b82800160010185558215610669579182015b8281111561066857825182559160200191906001019061064d565b5b509050610676919061067a565b5090565b61069c91905b80821115610698576000816000905550600101610680565b5090565b9056fea2646970667358221220ac3e4c6fc316ad5dd1a5502b34d1768df9bacf1482ff3220a458d6eddaea492564736f6c63430006010033";
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin, arguments: ["Hello from web3"]})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()


Overwriting src/Hello1Deploy.js


In [8]:
!node src/Hello1Deploy.js

Deploying the contract from 0x00ea5349121A17bDE34c1E2F5f29b6c4Dafc2dC1
hash: 0xbaf08a78a81b3ed8512ec4aeccd4589c40f47744dca50a6b5a62e2503ade319e
---> The contract deployed to: 0x1F4F57F902E53C442591717d2605B2d0ebaa2379


개인망을 띄우고, node로 Hello1Deploy.js를 실행하면 화면좌측에 보이듯이 transaction이 생성되고 그 hash값 이 주어진다.

우측 geth 단말에서 보듯이 대기중 transaction hash 값이 서로 일치한다.
마이닝을 하고 나면 contract 주소가 주어진다.

```
INFO [01-08|11:37:07.548] Submitted contract creation
fullhash=0x2ef00671e310f594dca636acbc352ce61463e78110e8098b0d483c8374643248
contract=0x96d8b65B4586B090840d6587C216D4A69071F405
```

![alt text](figures/sol_helloV1Web3DeployMining.png "deploy hello.sol using web3 with mining")

### REMIX에서 이벤트 확인하기

* Environment를 Javascript VM으로
* Deploy버튼 옆에 생성자 함수를 호출하기 위해 문자열 "Hello from web3"을 넣는다. 문자열에 따옴표를 잊지 않는다.
* Deploy버튼을 누르면 "Deployed Contracts" 아래, Hello1 객체가 나타난다. 괄호안 메모리는 실제 블록체인에 배포되지 않았다는 의미 (Javascript VM이므로)
* 속성 및 함수 버튼들을 눌러 결과가 출력되는지 보자.
* setHello() 함수 버튼을 누르면 화면 우측 하단을 잘 살펴보자. 특히 굵은 글씨 "logs"를 누르면 하단에 표가 펼쳐지고 이벤트 로그를 볼 수 있다.

![alt text](figures/sol_eventREMIX.png "geth download page")

### 단계 4: 사용

이벤트는 transaction log에 저장이 된다.
이벤트는 거래가 마이닝되어 블럭에 포함되어야만 발생된다.
트랙잭션이 pending되어 완료가 되지 않으면, 로그에 포함되지 않기 때문에 여기서 이벤틑가 발생하는지 지켜보는 경우 대기하게 된다.
단 이벤트가 발생하는 순서는 지켜진다.
단순하게 블록체인의 transaction이 아닌 ```call()```은 이벤트를 발생하지 않게 된다.

In [9]:
%%writefile src/Hello1Use.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8345"));
var _abiArray = [{"inputs":[{"internalType":"string","name":"_hello","type":"string"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"addr","type":"address"},{"indexed":false,"internalType":"string","name":"s","type":"string"}],"name":"PrintLog","type":"event"},{"inputs":[{"internalType":"string","name":"_str","type":"string"}],"name":"compareTo","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"sayHello","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"setHello","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var hello = new web3.eth.Contract(_abiArray, "0x1F4F57F902E53C442591717d2605B2d0ebaa2379");
var event = hello.events.PrintLog(function (error, result) {
    if (!error) {
        console.log("Event fired: " + JSON.stringify(result) + "\n---> " + JSON.stringify(result.returnValues));
    }
});

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    hello.methods.sayHello().call().then(console.log);  //null
    await hello.methods.setHello().send({from: accounts[0]});
    hello.methods.sayHello().call().then(console.log);
    hello.methods.compareTo("Hello").call().then(console.log);
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    
}
doIt()


Overwriting src/Hello1Use.js


In [10]:
!node src/Hello1Use.js

Account: 0x00ea5349121A17bDE34c1E2F5f29b6c4Dafc2dC1
Balance before: 99976385920000000000
Hello from web3
Hello
true
Balance after: 99975788900000000000
Balance diff: 597019999993856


## 블록체인에서 컨트랙 삭제

블록체인에서 프로그램을 유지보수할 수 없다. 오류가 발생하거나, 쓸모가 없어지면 삭제를 해야 한다.
컨트랙은 Read Only로서, 만들어지면 영구히 남게된다.
selfdestruct()는 잔고를 전액 반환하고, 컨트랙을 블록체인으로부터 제거한다.

###  단계 1: 컨트랙 개발

In [42]:
%%writefile src/Hello2.sol
pragma solidity ^0.6.0;
contract Hello2 {
    string hello;
    address payable owner;

    event PrintLog(string _s);
    constructor() public {
        owner = msg.sender;
    }
    function sayHello() view public returns(string memory) {
        return hello;
    }
    function setHello(string memory _hello) public payable {
        hello = _hello;
        emit PrintLog(_hello);
    }
    function getBalance() view public returns(uint) {
        return address(this).balance;
    }
    function kill() public { if (msg.sender == owner) selfdestruct(owner); }
}

Overwriting src/Hello2.sol


### 단계 2: 컴파일

In [1]:
!solc src/Hello2.sol --combined-json abi,bin > src/Hello2.json

In [44]:
!cat src/Hello2.json

{"contracts":{"src/Hello2.sol:Hello2":{"abi":"[{\"inputs\":[],\"stateMutability\":\"nonpayable\",\"type\":\"constructor\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"internalType\":\"string\",\"name\":\"_s\",\"type\":\"string\"}],\"name\":\"PrintLog\",\"type\":\"event\"},{\"inputs\":[],\"name\":\"getBalance\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"kill\",\"outputs\":[],\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"sayHello\",\"outputs\":[{\"internalType\":\"string\",\"name\":\"\",\"type\":\"string\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[{\"internalType\":\"string\",\"name\":\"_hello\",\"type\":\"string\"}],\"name\":\"setHello\",\"outputs\":[],\"stateMutability\":\"payable\",\"type\":\"function\"}]","bin":"608060405234801561001057600080fd5b5033600160006101000a81548173ffffffffffffffffffffffffff

### 단계 3: 배포

컴파일한 후 abi, bin을 저장한 파일에서 읽어서 배포해보자.


In [45]:
%%writefile src/Hello2DeployFromFile.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/Hello2.json");
var _json=JSON.parse(_str)
//var _abiArray=JSON.parse(_json.contracts.sHello2.abi);
var _abiArray=JSON.parse(_json.contracts['src/Hello2.sol:Hello2'].abi);
//var _bin=_json.contracts.sHello2.bin;
var _bin="0x"+_json.contracts['src/Hello2.sol:Hello2'].bin;

//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()


Overwriting src/Hello2DeployFromFile.js


In [64]:
!node src/Hello2DeployFromFile.js

Deploying the contract from 0x00ea5349121A17bDE34c1E2F5f29b6c4Dafc2dC1
hash: 0xb46dc1f7f644119fb6b053a78c1b508e707a1ee800e8e6017e850c3e7070f9b9
---> The contract deployed to: 0x25E943cA8788cc3EcED185E57f9bD94Ed96d342a


### 단계 4: 사용

In [65]:
%%writefile src/Hello2UseFromFile.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/Hello2.json");
var _json=JSON.parse(_str)
//var _abiArray=JSON.parse(_json.contracts.sHello2.abi);
var _abiArray=JSON.parse(_json.contracts['src/Hello2.sol:Hello2'].abi);

var hello = new web3.eth.Contract(_abiArray, "0x25E943cA8788cc3EcED185E57f9bD94Ed96d342a");
var event = hello.events.PrintLog(function (error, result) {
    if (!error) {
        console.log("Event fired: " + JSON.stringify(result) + "\n---> " + JSON.stringify(result.returnValues));
    }
});

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    hello.methods.sayHello().call().then(console.log);  //null
    await hello.methods.setHello("Hello World!").send({from: accounts[0], value: 1111});
    hello.methods.sayHello().call().then(console.log);
    //hello.methods.getBalance().call(function(err, bal) { console.log("Contract Balance: "+bal) });
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    //hello.methods.kill().send({from: accounts[0]})
}

doIt()


Overwriting src/Hello2UseFromFile.js


kill 함수를 마지막에 호출하기 때문에 한 번 실행해서 결과를 출력한 후, 재호출하면 ```Error: Returned values aren't valid, did it run Out of Gas?```가 호출된다.

In [66]:
!node src/Hello2UseFromFile.js

Account: 0x00ea5349121A17bDE34c1E2F5f29b6c4Dafc2dC1
Balance before: 99933124539999998889

Hello World!
Balance after: 99932213019999997778
Balance diff: 911520000000000


## 9.7 데이터구조

### 9.7.1 array

* 배열은 순서가 있는 데이터로서 인덱스를 사용한다.

#### 고정배열

고정배열은 배열이 저장하는 데이터의 개수를 사전에 정할 수 있는 경우에 사용한다.

```python
string[3] cities1=["Seoul", "Sydney", "Tokyo"];
```

괄호 안에 개수를 적어서 초기화할 수 있다.
```python
int[] memory num=new int[](3);
```

#### 동적배열

* 생성 Dynamic memory arrays are created using new keyword.

uint[] memory a = new uint[](7);



저장하는 데이터의 개수를 미리 정할 수 없고 나중에 정해진다.
```string```은 그 자체로 동적배열이다.
동적배열은 storage함수에만 사용할 수 있고, memory 변수에는 사용할 수 없다.
Storage 배열에는 push() 함수를 사용하여 데이터를 추가할 수 있다.

```python
string[] cities2;
cities2.push("New York");
cities2.push("Beijing");
```

* 다차원배열

정수의 동적 2차원배열이다.
앞의 3은 요소의 개수를, 뒤는 배열의 개수를 의미한다.
다음 예는 3개의 요소를 가진 배열이 2개라는 선언이다.

```python
uint[3][] marks=[[100, 80, 95],[20,30,40]];
```

* 배열의 반환
    * 상태변수는 반환을 할 수 있다.
    * **동적 배열은 그 크기를 알 수 없으므로 반환을 할 수 없다**. 배열을 반환하는 경우 사전에  크기를 알아야 한다.
    * array memory 반환 pragma experimental ABIEncoderV2;

* stat var를 변경하는 경우에는 ```view```라고 정의해서는 안된다.
* 삭제
```python
delete array[index];
```
// If we have data = [11, 12, 13, 14]
delete data[2];
// After deleting we have data = [11, 12, 0, 14]


* ```length```를 사용하여 배열 크기를 알 수 있다.
* ```push```는 ```storage``` 배열에만 사용하고 (즉 상태변수의 배열), ```memory``` 배열에는 사용할 수 없다.

## 실습 배열

* 아래 ```getMathAbove70()```는 REMIX에서는 실행되어 결과가 올바르게 출력되나, 배포한 후 함수를 호출하면 빈 배열만 출력된다.
* ```getCities()```는 동적배열을 반환하는 함수이다 (string의 배열이므로 동적 배열). 동적배열은 반환할 수 없다.

In [69]:
%%writefile src/ArrayTest2.sol
pragma solidity ^0.6.0;
pragma experimental ABIEncoderV2;

contract ArrayTest2 {
    string[3] cities1=["Seoul", "Sydney", "Tokyo"];
    string[] cities2 = new string[](2);
    string[] cities3;
    int[5] mathMarks;
    uint[3][] marks=[[100, 80, 95],[20,30,40]];

    function getDynamicArrMemory() pure public returns(uint[] memory) {
        uint[] memory num=new uint[](3);  //dynamic
        for (uint i=0; i<num.length; i++)
            num[i]=i;       //push() not allowed for array memeory
        return num;
    }
    //string is a dynamic array itself, which can not be returned.
    //function setArrMemory() view public returns(string[] memory) {
    function getStringDynamicArrMemory() pure public {
        //arr storage in memory not allowed
        //error: string[2] places = ["9000", "Sydney"];
        string[2] memory places = ["9000", "Sydney"];
        //array memory push not allowed
        //places.push("Seoul");
        places[0]="Seoul";
        //return places;
    }
    /*function getCities1_() view public returns(string[] memory) {
        return cities1;  //can not return stoarge var. 
    }*/
    function getCities1() view public returns(string memory) {
        return cities1[0];
    }
    function getCities1Length() view public returns(uint) { return cities1.length; }
    function setCities23() public {
        cities2[0]="New York";
        cities2.push("Busan");
        cities3.push("New York");
        cities3.push("Beijing");
    }
    //dynamic array return needs "pragma experimental ABIEncoderV2;"
    function getCities2() view public returns(string[] memory){
        return cities2;
    }
    function setMathMarks() public {
        mathMarks=[100,60,95,50,80];
    }
    //run setMathMarks() beforehand
    function getMathAbove70() view public returns(int[] memory) {
        int[] memory mathAbove70;   // size is not allocated yet
        uint counter = 0;
        for(uint8 i=0;i<mathMarks.length;i++)
            if(mathMarks[i]>70) {
                mathAbove70[counter] = mathMarks[i];
                //mathAbove70.push(mathMarks[i]);
                counter++;
            }
        return mathAbove70;
    }
    function getMathAbove70_() view public returns(int[] memory) {
        //compute lengthOfMathAbove70
        uint8 counter=0;
        uint8 lengthOfMathAbove70=0;
        for(uint8 i=0;i<mathMarks.length;i++)
            if(mathMarks[i]>70) counter++;
        lengthOfMathAbove70=counter;
        //move math marks above 70
        int[] memory mathAbove70=new int[](lengthOfMathAbove70);
        counter=0;
        for(uint i=0;i<mathMarks.length;i++) {
            if(mathMarks[i]>70) {
                mathAbove70[counter]=mathMarks[i];
                counter++;
            }
        }
        return mathAbove70;
    }
    function updateMarks() public returns(uint[3][] memory){
        marks[0][0]=90;
        return marks;
    }
    function getMarksArr() view public returns(uint[3][] memory) {
        return marks;
    }
    function getMarksLength() view public returns(uint) {
        return marks.length;
    }
}

Writing src/ArrayTest2.sol


In [ ]:
!solc --abi --bin src/ArrayTest2.sol

In [ ]:
# %load src/ArrayTestDeploy.js
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
}
var _abiArray=[{"constant":false,"inputs":[],"name":"setEngMarks","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getMarksArr","outputs":[{"name":"","type":"uint256[3][]"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"getMathAbove70Storage","outputs":[{"name":"","type":"int256[]"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"getMathAbove70","outputs":[{"name":"","type":"int256[]"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getArrMemory","outputs":[{"name":"","type":"uint256[]"}],"payable":false,"stateMutability":"pure","type":"function"},{"constant":false,"inputs":[],"name":"setCities","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getMarksLength","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"setMathMarks","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getEngMarks","outputs":[{"name":"","type":"int256[]"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"updateMarks","outputs":[{"name":"","type":"uint256[3][]"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getDynamicArrMemory","outputs":[],"payable":false,"stateMutability":"pure","type":"function"},{"constant":true,"inputs":[],"name":"getCities2","outputs":[{"name":"","type":"string[]"}],"payable":false,"stateMutability":"view","type":"function"}];
var _bin="608060405260405180604001604052806040518060600160405280606460ff168152602001605060ff168152602001605f60ff1681525081526020016040518060600160405280601460ff168152602001601e60ff168152602001602860ff16815250815250600690600262000077929190620001a9565b5060405180606001604052806040518060400160405280600581526020017f53656f756c00000000000000000000000000000000000000000000000000000081525081526020016040518060400160405280600681526020017f5379646e6579000000000000000000000000000000000000000000000000000081525081526020016040518060400160405280600581526020017f546f6b796f0000000000000000000000000000000000000000000000000000008152508152506008906003620001449291906200020e565b5060026040519080825280602002602001820160405280156200017c57816020015b6060815260200190600190039081620001665790505b50600b90805190602001906200019492919062000268565b50348015620001a257600080fd5b506200048b565b828054828255906000526020600020906003028101928215620001fb579160200282015b82811115620001fa57825182906003620001e9929190620002cf565b5091602001919060030190620001cd565b5b5090506200020a919062000319565b5090565b826003810192821562000255579160200282015b8281111562000254578251829080519060200190620002439291906200034a565b509160200191906001019062000222565b5b509050620002649190620003d1565b5090565b828054828255906000526020600020908101928215620002bc579160200282015b82811115620002bb578251829080519060200190620002aa9291906200034a565b509160200191906001019062000289565b5b509050620002cb9190620003d1565b5090565b826003810192821562000306579160200282015b8281111562000305578251829060ff16905591602001919060010190620002e3565b5b50905062000315919062000402565b5090565b6200034791905b808211156200034357600081816200033991906200042a565b5060030162000320565b5090565b90565b828054600181600116156101000203166002900490600052602060002090601f016020900481019282601f106200038d57805160ff1916838001178555620003be565b82800160010185558215620003be579182015b82811115620003bd578251825591602001919060010190620003a0565b5b509050620003cd919062000402565b5090565b620003ff91905b80821115620003fb5760008181620003f191906200043f565b50600101620003d8565b5090565b90565b6200042791905b808211156200042357600081600090555060010162000409565b5090565b90565b50600081556001016000815560010160009055565b50805460018160011615610100020316600290046000825580601f1062000467575062000488565b601f01602090049060005260206000209081019062000487919062000402565b5b50565b610fc1806200049b6000396000f3fe608060405234801561001057600080fd5b50600436106100b45760003560e01c8063a7ad780e11610071578063a7ad780e14610145578063a8bdea1d14610163578063aa2025d31461016d578063cec11e681461018b578063d8260e20146101a9578063f57650a4146101b3576100b4565b80632bbcb02d146100b9578063479998e7146100c357806362ceacb6146100e15780637a551ab2146100ff5780638e435ee81461011d578063a77d71771461013b575b600080fd5b6100c16101d1565b005b6100cb610200565b6040516100d89190610d56565b60405180910390f35b6100e9610290565b6040516100f69190610d78565b60405180910390f35b6101076102e8565b6040516101149190610d78565b60405180910390f35b61012561041f565b6040516101329190610dbc565b60405180910390f35b610143610492565b005b61014d610646565b60405161015a9190610dde565b60405180910390f35b61016b610653565b005b6101756106a1565b6040516101829190610d78565b60405180910390f35b6101936106f9565b6040516101a09190610d56565b60405180910390f35b6101b16107b8565b005b6101bb61088f565b6040516101c89190610d9a565b60405180910390f35b600560649080600181540180825580915050906001820390600052602060002001600090919290919091505550565b60606006805480602002602001604051908101604052809291908181526020016000905b828210156102875783829060005260206000209060030201600380602002604051908101604052809291908260038015610273576020028201915b81548152602001906001019080831161025f575b505050505081526020019060010190610224565b50505050905090565b606060078054806020026020016040519081016040528092919081815260200182805480156102de57602002820191906000526020600020905b8154815260200190600101908083116102ca575b5050505050905090565b60606000809050600080905060008090505b60058160ff16101561033657604660008260ff166005811061031857fe5b015413156103295782806001019350505b80806001019150506102fa565b5081905060608160ff1660405190808252806020026020018201604052801561036e5781602001602082028038833980820191505090505b5090506000925060008090505b60058110156104155760466000826005811061039357fe5b0154131561040857600081600581106103a857fe5b0154828560ff16815181106103b957fe5b6020026020010181815250506007600082600581106103d457fe5b0154908060018154018082558091505090600182039060005260206000200160009091929091909150555083806001019450505b808060010191505061037b565b5080935050505090565b60608060036040519080825280602002602001820160405280156104525781602001602082028038833980820191505090505b50905060008090505b815181101561048a578082828151811061047157fe5b602002602001018181525050808060010191505061045b565b508091505090565b6040518060400160405280600881526020017f4e657720596f726b000000000000000000000000000000000000000000000000815250600b6000815481106104d657fe5b9060005260206000200190805190602001906104f3929190610978565b50600b8060018154018082558091505090600182039060005260206000200160006040518060400160405280600581526020017f427573616e00000000000000000000000000000000000000000000000000000081525090919091509080519060200190610562929190610978565b5050600c8060018154018082558091505090600182039060005260206000200160006040518060400160405280600881526020017f4e657720596f726b000000000000000000000000000000000000000000000000815250909190915090805190602001906105d2929190610978565b5050600c8060018154018082558091505090600182039060005260206000200160006040518060400160405280600781526020017f4265696a696e670000000000000000000000000000000000000000000000000081525090919091509080519060200190610642929190610978565b5050565b6000600680549050905090565b6040518060a00160405280606460ff168152602001603c60ff168152602001605f60ff168152602001603260ff168152602001605060ff16815250600090600561069e9291906109f8565b50565b606060058054806020026020016040519081016040528092919081815260200182805480156106ef57602002820191906000526020600020905b8154815260200190600101908083116106db575b5050505050905090565b6060605a600660008154811061070b57fe5b906000526020600020906003020160006003811061072557fe5b01819055506006805480602002602001604051908101604052809291908181526020016000905b828210156107af578382906000526020600020906003020160038060200260405190810160405280929190826003801561079b576020028201915b815481526020019060010190808311610787575b50505050508152602001906001019061074c565b50505050905090565b6107c0610a3d565b60405180604001604052806040518060400160405280600481526020017f393030300000000000000000000000000000000000000000000000000000000081525081526020016040518060400160405280600681526020017f5379646e6579000000000000000000000000000000000000000000000000000081525081525090506040518060400160405280600581526020017f53656f756c0000000000000000000000000000000000000000000000000000008152508160006002811061088457fe5b602002018190525050565b6060600b805480602002602001604051908101604052809291908181526020016000905b8282101561096f578382906000526020600020018054600181600116156101000203166002900480601f01602080910402602001604051908101604052809291908181526020018280546001816001161561010002031660029004801561095b5780601f106109305761010080835404028352916020019161095b565b820191906000526020600020905b81548152906001019060200180831161093e57829003601f168201915b5050505050815260200190600101906108b3565b50505050905090565b828054600181600116156101000203166002900490600052602060002090601f016020900481019282601f106109b957805160ff19168380011785556109e7565b828001600101855582156109e7579182015b828111156109e65782518255916020019190600101906109cb565b5b5090506109f49190610a64565b5090565b8260058101928215610a2c579160200282015b82811115610a2b578251829060ff16905591602001919060010190610a0b565b5b509050610a399190610a89565b5090565b60405180604001604052806002905b6060815260200190600190039081610a4c5790505090565b610a8691905b80821115610a82576000816000905550600101610a6a565b5090565b90565b610aab91905b80821115610aa7576000816000905550600101610a8f565b5090565b90565b6000610aba8383610c3b565b60608301905092915050565b6000610ad28383610cf0565b60208301905092915050565b6000610aea8383610cff565b905092915050565b6000610afe8383610d38565b60208301905092915050565b6000610b1582610e43565b610b1f8185610ec6565b9350610b2a83610df9565b8060005b83811015610b5b578151610b428882610aae565b9750610b4d83610e85565b925050600181019050610b2e565b5085935050505092915050565b6000610b7382610e4e565b610b7d8185610ed7565b9350610b8883610e09565b8060005b83811015610bb9578151610ba08882610ac6565b9750610bab83610e92565b925050600181019050610b8c565b5085935050505092915050565b6000610bd182610e59565b610bdb8185610ee8565b935083602082028501610bed85610e19565b8060005b85811015610c295784840389528151610c0a8582610ade565b9450610c1583610e9f565b925060208a01995050600181019050610bf1565b50829750879550505050505092915050565b610c4481610e64565b610c4e8184610ef9565b9250610c5982610e29565b8060005b83811015610c8a578151610c718782610af2565b9650610c7c83610eac565b925050600181019050610c5d565b505050505050565b6000610c9d82610e6f565b610ca78185610f04565b9350610cb283610e33565b8060005b83811015610ce3578151610cca8882610af2565b9750610cd583610eb9565b925050600181019050610cb6565b5085935050505092915050565b610cf981610f26565b82525050565b6000610d0a82610e7a565b610d148185610f15565b9350610d24818560208601610f3a565b610d2d81610f6d565b840191505092915050565b610d4181610f30565b82525050565b610d5081610f30565b82525050565b60006020820190508181036000830152610d708184610b0a565b905092915050565b60006020820190508181036000830152610d928184610b68565b905092915050565b60006020820190508181036000830152610db48184610bc6565b905092915050565b60006020820190508181036000830152610dd68184610c92565b905092915050565b6000602082019050610df36000830184610d47565b92915050565b6000819050602082019050919050565b6000819050602082019050919050565b6000819050602082019050919050565b6000819050919050565b6000819050602082019050919050565b600081519050919050565b600081519050919050565b600081519050919050565b600060039050919050565b600081519050919050565b600081519050919050565b6000602082019050919050565b6000602082019050919050565b6000602082019050919050565b6000602082019050919050565b6000602082019050919050565b600082825260208201905092915050565b600082825260208201905092915050565b600082825260208201905092915050565b600081905092915050565b600082825260208201905092915050565b600082825260208201905092915050565b6000819050919050565b6000819050919050565b60005b83811015610f58578082015181840152602081019050610f3d565b83811115610f67576000848401525b50505050565b6000601f19601f830116905091905056fea365627a7a723058208381cfc68cbd85a2542cf4949b3b83a647248c24cdcd850a6a0a56ccd49306016c6578706572696d656e74616cf564736f6c63430005090040";
var _contract = web3.eth.contract(_abiArray);
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
var _instance=_contract.new({data:"0x"+_bin,from:web3.eth.accounts[0],gas:1000000}, function(err, contract) {
    if (!err) {
        console.log("contractAddress: ", contract.address);
        console.log("transactionHash: ", contract.transactionHash);
    }
});

In [ ]:
!node src/ArrayTestDeploy.js

In [ ]:
### arraytest use

* ```getMathAbove70()``` 함수는 빈 배열을 출력하고 있어 문제가 있다.

In [ ]:
%%writefile src/ArrayTestUse.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8545"));
var arraytestContract = web3.eth.contract([{"constant":true,"inputs":[],"name":"getMarksArr","outputs":[{"name":"","type":"uint256[3][]"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"getMathAbove70","outputs":[{"name":"","type":"int256[]"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"getArrMemory","outputs":[{"name":"","type":"uint256[]"}],"payable":false,"stateMutability":"pure","type":"function"},{"constant":true,"inputs":[],"name":"getCities","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"setCities","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getMarksLength","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"setMathMarks","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"updateMarks","outputs":[{"name":"","type":"uint256[3][]"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getDynamicArrMemory","outputs":[],"payable":false,"stateMutability":"pure","type":"function"}]);
var arraytest = arraytestContract.at("0xc5a134071d8d9d64f582d88ece87d384ab1fe68b");
console.log(arraytest.setMathMarks({from:web3.eth.accounts[0]}));
console.log(arraytest.getMathAbove70());
console.log(arraytest.getArrMemory().toString());
console.log(arraytest.getMarksArr().toString());

In [ ]:
```
0x9e05bb4d68e98b728f2acfb9eeb22b1101687b92e1dfa410ddbc0cbeb27a4fcb
[]
0,1,2
100,80,95,20,30,40
```

```python
> arraytest.getMarksArr()                      출력하기 위해서는 toString()
[ [ BigNumber { s: 1, e: 2, c: [Array] },
    BigNumber { s: 1, e: 1, c: [Array] },
    BigNumber { s: 1, e: 1, c: [Array] } ],
  [ BigNumber { s: 1, e: 1, c: [Array] },
    BigNumber { s: 1, e: 1, c: [Array] },
    BigNumber { s: 1, e: 1, c: [Array] } ] ]
> arraytest.getMarksLength()
BigNumber { s: 1, e: 0, c: [ 2 ] }
> arraytest.getMarksLength().toString()
'2'
> arraytest.getMarksArr().toString()
'100,80,95,20,30,40'
> arraytest.getArrMemory();
[ BigNumber { s: 1, e: 0, c: [ 0 ] },
  BigNumber { s: 1, e: 0, c: [ 1 ] },
  BigNumber { s: 1, e: 0, c: [ 2 ] } ]
> arraytest.getArrMemory().toString();
'0,1,2'
> arraytest.getArrMemory().pop().toString()
'2'
```

배열을 검색하는 것은 비용이 많이 발생한다. 
mapping을 사용하거나, 클라이언트 측에서 배열 데이터를 검색하는 편으로 하자.

문자열 비교 '=='는 사용할 수 없다. ```keccak256()``` 해싱을 한 후 비교해야 한다.

In [2]:
%%writefile src/Members.sol
pragma solidity ^0.5.0;
contract Members{
    address owner;
    event printAddr(address arg);
    struct Member{
        uint id;
        string name;
    }
    Member[] public memberArr;
    constructor() public {
        owner=msg.sender;
    }
    function del() public {
        delete memberArr;
    }
    function delOne(uint i) public{
        delete memberArr[i];
    }
    function add(uint id,string memory name) public {
        memberArr.push(Member(id,name));
    }
    //return Member
    function getMember(string memory who) view public returns(uint, string memory) {
        uint _id;
        string memory _name;
        for(uint i=0;i<memberArr.length;i++) {
            _name=memberArr[i].name;
            if(keccak256(abi.encodePacked(_name))==keccak256(abi.encodePacked(who))) {
                _id=memberArr[i].id;
                _name=memberArr[i].name;
            }
        }
        return (_id,_name);
    }
    function compareStr(string memory s1, string memory s2) pure public returns(bool) {
        return keccak256(abi.encodePacked(s1))==keccak256(abi.encodePacked(s2));
    }
    function compareBytes(bytes memory b1, bytes memory b2) pure public returns(bool) {
        return keccak256(b1) == keccak256(b2);
    }
    function getLength() view public returns(uint) {
        return memberArr.length;
    }
}

Writing src/Members.sol


### 9.7.2 mapping
<a id='S.5.2'></a>

```mapping```은 `키`와 키에 해당하는 `값`을 쌍으로 저장한다. ```=>``` 기호를 사용하여, 좌측에 키를 오른쪽에 값을 적어준다.
아래는 (이름, 잔고)를 저장하는 mapping이다.

```python
mapping (string => uint) public balances;
balances["jsl"] = 100;
```

## 실습: 은행

앞서 은행에서는 한 계정에서의 입금, 출금을 실행해 보았다.
여러 계정이 있는 경우, mapping을 이용해 잔고를 관리할 수 있다.

```mapping(address=>uint) balanceOf;``` 라고 선언하고
```balanceOf[<address>]=amount;```로 잔고를 저장할 수 있다.

증액, 감액을 하려면:
```python
uint balanceToAdd=111;
balanceOf[<address>] += balanceToAdd;
```

* adddress 자신의 주소를 따옴표 없이, 금액도 따옴표 없이 정수

In [1]:
%%writefile src/BankV4.sol
//check for Coin.sol (ch12) and BankV3.sol 201906
pragma solidity ^0.5.0;
contract BankV3 {
    address minter;
    mapping (address => uint) balanceOf;
    constructor() public {
        minter = msg.sender;
    }
    function mint(address owner, uint amount) public {
        require(msg.sender == minter);
        balanceOf[owner] += amount;
    }
    function send(address receiver, uint amount) public {
        require(balanceOf[msg.sender]>=_value);
        balanceOf[msg.sender] -= amount; // Subtract from the sender
        balanceOf[receiver] += amount;   // Add the same to the recipient
    }
    function queryBalance(address addr) public view returns (uint) {
        return balanceOf[addr];
    }
}


Writing src/BankV4.sol


* 중복키 확인

맵은 그 특성상 중복키는 허용되지 않는다. 동일한 키로 당연히 데이터를 입력할 수가 없다.
맵에 이미 키가 존재하는지 확인할 수 있는 ```containsKey()```와 같은 지원 함수가 없다.
필드 ```isMember```를 설정해놓고 그 값을 확인하는 방법으로 하자.
```
mapping(address=>Member) memberMap;
struct Member {
	uint id;
	string name;
	bool isMember;
}
if (memberMap(<address>).isMember) 중복키 존재함
```

* 삭제

키삭제는 쉽지 않다. 다른 언어에서는 ```remove()```함수가 지원되어, 간편하게 키를 지울 수 있는 것과 비교된다.

```
delete pendingAccessRequest[key];
```

#### bidirectional map

주소에 대해 회원(아이디, 이름)을 저장하는 ```map```이다.
이 경우 '이름'으로 '주소'를 검색하는 기능은 ```bidiretional map```으로 구현하였다.

줄 | 설명
-----|-----
9 | 주소의  ```Member strcut ```을 조회하는 map
10 | 이름의 주소를 조회하는 map
11~16 ```addMember() ``` |  ```Member struct```을 추가하고 ```getMemberByName()``` 함수에서 이름을 입력하면 주소를 조회할 수 있도록 저장
17~19 ```getMemberByName()``` | bidirectional map에서 이름에 해당하는 주소를 조회
20~23 ```getMember()``` | 주소에 해당하는 ```Member struct```을 조회

In [1]:
%%writefile src/MembersMap.sol
pragma solidity ^0.5.0;

contract MembersMap {
    struct Member {
        uint id;
        string name;
    }
    //bidrectional map
    mapping(address=>Member) memberMap;
    mapping(string=>address) addressByName;
    function addMember (uint _id, string memory _name) public {
       Member memory m=Member(_id, _name);
       memberMap[msg.sender]=m;
       //saving into a bidiretional map to get address by name
       addressByName[_name]=msg.sender;
    }
    function getMemberByName(string memory _name) public view returns(address) {
        return addressByName[_name];
    }
    function getMember(address addr) view public returns (uint, string memory) {
        Member memory m=memberMap[addr];
        return (m.id, m.name);
    }
}

Writing src/MembersMap.sol


## 문제 주문

* 사용자별 주문내역 (1:n 관계)
* 주문id 중복 검사를 추가해야


줄 | 설명
-----|-----
4~12 | 주문 데이터, ```isOrdered```는 중복키 확인용도로 쓰인다.
14 | 주문 갯수. 반복문으로 갯수를 세는 것을 피하기 위해서 계속 추적한다.
15 | 주문 총액 역시 반복문을 피하기 위해 추적한다.
18 | 주문번호에서 검색을 하기 위해 설정한다.
22~30 ```placeOrder()``` | 주문금액 확인, 전송자 주소로 주문데이터 확인, 주문갯수 및 주문총액을 추적.
32~35 ```updateStatus()``` | 주문번호를 이용해 주문의 상태를 갱신한다. 관리자만 갱신권한을 가진다.


In [4]:
%%writefile src/Order.sol
pragma solidity ^0.5.0;

contract Order {
    struct OrderItem {
        uint id;
        string product;
        uint n; //number of items
        uint amount;
        uint time;
        string status;
        bool isOrdered; // check if exists
    }
    address manager;
    uint nOrder;    //track number of orderItems (to avoid for loop)
    uint totalOrderAmount; //track number of orderItems (to avoid for loop)
    //bidrectional map
    mapping(address=>OrderItem) orderItems;
    mapping(uint=>address) addressById;
    constructor() public {
        manager=msg.sender;
    }
    function placeOrder (uint _id, string memory _p, uint _n, uint _amount) public payable {
        require(msg.value==_amount);
        OrderItem memory o=OrderItem(_id, _p, _n, _amount, now, "Ordered", true);
        orderItems[msg.sender]=o;
        //saving into a bidiretional map to get address by id
        addressById[_id]=msg.sender;
        nOrder++;
        totalOrderAmount+=_amount;
    }
    //only manager allowed
    function updateStatus(uint _id, string memory _s) public isOwner returns(string memory) {
        orderItems[addressById[_id]].status=_s;
        return (orderItems[msg.sender].status);
    }
    function getStatus() view public returns(string memory) {
        return orderItems[msg.sender].status;
    }
    function getOrderById(uint _id) public view returns(uint,string memory,string memory) {
        OrderItem memory o=orderItems[addressById[_id]];
        return (o.id,o.product,o.status);
    }
    function getOrderItem() view public returns(uint,string memory,string memory) {
        OrderItem memory o=orderItems[msg.sender];
        return (o.id,o.product,o.status);
    }
    function getNOrder() view public returns(uint) {
        return nOrder;
    }
    function getTotalOrderAmount() view public returns(uint) {
        return totalOrderAmount;
    }
    function queryBalance() view public returns(uint) {
        return address(this).balance;
    }
    modifier isOwner() {
        require(msg.sender == manager);
        _;
    }
}

Overwriting src/Order.sol


## 9.8 Exception
<a id='S.6'></a>

오류는 발생하기 마련이다. 컴파일 오류는 수정하기 비교적 용이하다. 그러나 실행시점에 발생하는 오류는 잡아내기 어렵다.
다른 언어에서 보통 제공하는 ```try..catch``` 구문도 제공되지 않는다.
버전 4.10 이전에는 ```throw``` 명령문이 사용되었지만 문제가 있었다.
지급한 gas를 다 소비하고 오류발생 이전의 상태로 돌려놓았다.

```python
if (msg.sender != owner) { throw; }
```

버전 4.10이후에는 ```throw```구문은 더 이상 사용되지 않고 다음 구문이 사용된다.


예외는 배열이 범위를 넘어서는 경우와 같이 실행되면서 발생할 수 있다.
반면에 ```out-of-gas errors```, ```divide by zero```와 같이 어떤 특정 조건에서 예외를 발생시킬 수 있다.
이 경우 실행이 중지되고, 원상복구 (상태변수와 잔고)된다.

* ```assert()``` 미사용 gas는 반환되지 않고, 모두 소모된다. 상태는 원래대로 복원된다.
* ```require()``` | ```true```, ```false```로 반환된다. ```false```인 경우 예외가 발생하고 실행이 중지된다. 미사용 gas를 호출자에게 반환하고 상태를 원래대로 복원한다.
* ```revert()```  ```require()``` 함수와 동일한 의미이지만, 어떤 평가를 하지 않는다는 점이 다르다. 즉, ```revert`` 명령문은 미사용 gas를 호출자에게 반환하고 상태를 원래대로 복원한다.

명령문 | gas 반환 | 원복 | 사용하는 경우
-----|-----|-----|-----
```assert()``` | NO | YES | overflow/underflow, invariants 등 어떤 오류가 발생하는 것을 예방하는 경우
```revert()``` | YES | YES | ```require()```는 참, 거짓을 평가하지만 조건문 없이 수행되기 때문에 이진비교 이상을 하는 경우
```require()``` | YES | YES | 사용자 입력 , 상태변수 값, 반환 값 등 조건을 검증하는 경우

In [3]:
%%writefile src/ExceptionTest.sol
pragma solidity *0.5.0;

contract ExceptionTest {
    address owner;
    constructor() public {
        owner=msg.sender;
    }
    function requireException() view public returns(string memory) {
        //if (msg.sender != owner) { throw; }
        require(msg.sender != owner, "Sorry! You are NOT owner. Require failed...");
        return "asserted";
    }
    function assertException() view public returns(string memory) {
        assert(msg.sender == owner);
        return "asserted";
    }
    function revertException() view public returns(string memory) {
        if(msg.sender != owner) {
            revert("Sorry! You are NOT owner. Reverted...");
            return "reverted";
        } else {
            return "not reverted";
        }
    }
    function raiseException() view public {
        int x=0;
        int y=0;
        x/y;    //divide by zero
    }
}

Overwriting src/ExceptionTest.sol


## 실습

A, B 2인의 가위바위보 게임 후 게임머니를 이긴 사람에게 지급

In [2]:
%%writefile src/Rsp.sol
pragma solidity ^0.5.0;

contract Rsp {
    uint8 betA; //1 scissor, 2 rock or 3 paper
    uint8 betB; //1 scissor, 2 rock or 3 paper
    string matchResult; //win or lose message
    bytes32 winnerAorB;  //bytes32 enables the '==' comparison 
    //string winnerAorB;  //A or B
    uint betAmount;     //smaller bet between A and B
    mapping(address => uint256) public balanceOf;
    mapping(string => address) playersAddr;
    //mapping(bytes32 => address) public playersAddr;
    
    event PrintLog(string);
    //event PrintLog(bytes32);
    function setA(uint8 n, uint amount) public payable {
        // ADD: check if 'n' is 1,2 or 3
        betA=n;
        betAmount=amount;
        depositBalance(betAmount);
        addPlayersAddr("A");
    }
    function setB(uint8 n, uint amount) public payable {
        betB=n;
        betAmount=amount;   // bets of A and B to be the same
        depositBalance(betAmount);
        addPlayersAddr("B");
    }
    function play() public {
        if(betA==betB) {
            matchResult="tie";
            winnerAorB="N";
        } else if(betA==1 && betB==2) {
            matchResult="B wins";
            winnerAorB="B";
        } else if(betA==1 && betB==3) {
            matchResult="A wins";
            winnerAorB="A";
        } else if(betA==2 && betB==1) {
            matchResult="A wins";
            winnerAorB="A";
        } else if(betA==2 && betB==3) {
            matchResult="B wins";
            winnerAorB="B";
        } else if(betA==3 && betB==1) {
            matchResult="B wins";
            winnerAorB="B";
        } else if(betA==3 && betB==2) {
            matchResult="A wins";
            winnerAorB="A";
        }
    }
    function getMatchResult() view public returns(string memory) {
        return matchResult;
    }
    function distributeBetAmount() public {
        if (winnerAorB=="A") {
            // deduct from the loser and give all to ther winner
            balanceOf[playersAddr["A"]]+=betAmount;
            balanceOf[playersAddr["B"]]-=betAmount;
        } else if (winnerAorB=="B") {
            // deduct from the loser and betAmountgive all to ther winner
            balanceOf[playersAddr["B"]]+=betAmount;
            balanceOf[playersAddr["A"]]-=betAmount;
        }
    }
    function queryBalance() view public returns(uint) {
        return address(this).balance;
    }
    function queryBalanceOf(string memory player) view public returns(uint) {
        //ADD: check a to be 'A' or 'B'
        return balanceOf[playersAddr[player]];
    }
    function depositBalance(uint amount) public payable {
        require(msg.value==amount);
        // ADD: check if msg.sender exists
        balanceOf[msg.sender]+=amount;        
    }
    function addPlayersAddr(string memory p) internal {
        // ADD: check if p exists
        playersAddr[p]=msg.sender;
    }
}

Writing src/Rsp.sol
